In [22]:
# Import minimal requirements
import requests
import json
import re

In [23]:
# Set the base URL for the ARAX reasoner and its endpoint
endpoint_url = 'https://arax.rtx.ai/devED/api/rtx/v1/query'

# Create a dict of the request, specifying the list of DSL commands
query = { "previous_message_processing_plan": { "processing_actions": [
            "add_qnode(curie=DOID:14330, id=n00)",
            "add_qnode(type=protein, is_set=true, id=n01)",
            "add_qnode(type=chemical_substance, id=n02)",
            "add_qedge(source_id=n00, target_id=n01, id=e00)",
            "add_qedge(source_id=n01, target_id=n02, id=e01, type=physically_interacts_with)",
            "expand(edge_id=[e00,e01], kp=ARAX/KG1)",
            "overlay(action=compute_jaccard, start_node_id=n00, intermediate_node_id=n01, end_node_id=n02, virtual_edge_type=J1)",
            "filter_kg(action=remove_edges_by_attribute, edge_attribute=jaccard_index, direction=below, threshold=.2, remove_connected_nodes=t, qnode_id=n02)",
            "filter_kg(action=remove_edges_by_property, edge_property=provided_by, property_value=Pharos)",
            "overlay(action=predict_drug_treats_disease, source_qnode_id=n02, target_qnode_id=n00, virtual_edge_type=P1)", 
            "resultify(ignore_edge_direction=true)",
            "filter_results(action=sort_by_edge_attribute, edge_attribute=probability_drug_treats, direction=descending, max_results=15)", 
            "return(message=true, store=true)",
            ] } }

In [24]:
# Send the request to RTX and check the status
print(f"Executing query at {endpoint_url}\nPlease wait...")
response_content = requests.post(endpoint_url, json=query, headers={'accept': 'application/json'})
status_code = response_content.status_code
if status_code != 200:
    print("ERROR returned with status "+str(status_code))
    print(response_content.json())
else:
    print(f"Response returned with status {status_code}")

Executing query at https://arax.rtx.ai/devED/api/rtx/v1/query
Please wait...
Response returned with status 200


In [25]:
# Unpack respsonse from JSON and display the information log
response_dict = response_content.json()
for message in response_dict['log']:
    if message['level'] >= 20:
        print(message['prefix']+message['message'])

2020-03-13 04:12:44.453937 INFO: ARAXQuery launching
2020-03-13 04:12:44.453960 INFO: Examine input query for needed information for dispatch
2020-03-13 04:12:44.453969 INFO: Found input processing plan. Sending to the ProcessingPlanExecutor
2020-03-13 04:12:44.456072 INFO: Parsing input actions list
2020-03-13 04:12:44.456608 INFO: Adding a QueryNode to Message with parameters {'id': 'n00', 'curie': 'DOID:14330', 'name': None, 'type': None, 'is_set': None}
2020-03-13 04:12:44.457000 INFO: Adding a QueryNode to Message with parameters {'id': 'n01', 'curie': None, 'name': None, 'type': 'protein', 'is_set': 'true'}
2020-03-13 04:12:44.457111 INFO: Adding a QueryNode to Message with parameters {'id': 'n02', 'curie': None, 'name': None, 'type': 'chemical_substance', 'is_set': None}
2020-03-13 04:12:44.457213 INFO: Adding a QueryEdge to Message with parameters {'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': None}
2020-03-13 04:12:44.457247 INFO: Adding a QueryEdge to Message w

In [26]:
# These URLs provide direct access to resulting data and GUI
if 'id' in response_dict and response_dict['id'] is not None:
    print(f"Data: {response_dict['id']}")
    match = re.search(r'(\d+)$', response_dict['id'])
    if match:
        print(f"GUI: https://arax.rtx.ai/devLM/?m={match.group(1)}")
else:
    print("No id was returned in response")

Data: https://arax.rtx.ai/api/rtx/v1/message/1451
GUI: https://arax.rtx.ai/devLM/?m=1451


In [27]:
# Or you can view the entire Translator API response Message
print(json.dumps(response_dict, indent=2, sort_keys=True))

{
  "code_description": "Normal completion",
  "context": "https://raw.githubusercontent.com/biolink/biolink-model/master/context.jsonld",
  "datetime": "2020-03-13 04:12:44",
  "id": "https://arax.rtx.ai/api/rtx/v1/message/1451",
  "knowledge_graph": {
    "edges": [
      {
        "id": "1346604",
        "is_defined_by": "RTX",
        "provided_by": "BioLink",
        "qedge_id": "e00",
        "relation": "associated_with_disease",
        "source_id": "UniProtKB:P02675",
        "target_id": "DOID:14330",
        "type": "gene_associated_with_condition"
      },
      {
        "id": "1357727",
        "is_defined_by": "RTX",
        "provided_by": "DisGeNet",
        "qedge_id": "e00",
        "relation": "gene_associated_with_condition",
        "source_id": "UniProtKB:P01375",
        "target_id": "DOID:14330",
        "type": "gene_associated_with_condition"
      },
      {
        "id": "1338488",
        "is_defined_by": "RTX",
        "provided_by": "BioLink",
        "q